### Load Packages

In [1]:
import pandas as pd
import numpy as np
import os

pd.options.mode.chained_assignment = None

### Import Excel Files

In [2]:
folder = r'C:\Users\mazhang\Documents\Jupyter Notebook\2021 Weekly Closed Invoice\Closed_invoice_01_03_2021'
files = os.listdir(folder)

excel_list = []
for file in files:
    if file.endswith('.xlsx'):
        df = pd.read_excel(os.path.join(folder, file), sheet_name = None)
        excel_list.append(df)

C:\Users\mazhang\Anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


### Generate Weekly Report with Accounting ID, Facility and  Bill Pay Category, etc.

In [56]:
df_list = []

for excel in range(len(excel_list)):
    
    df_header = excel_list[excel]['Header']
    df_line = excel_list[excel]['Line']

    df_select_header = df_header[['Accounting_CustomerID', 'CustomerID', 'DocNo', 'Facility', 'Bill To', 'InvoiceDate']]
    df_drop_dup_header = df_select_header.drop_duplicates(subset = ['Accounting_CustomerID', 'DocNo',
                                                                    'Facility', 'Bill To'])
    
    df_select_line = df_line[['CustomerID', 'Facility', 'Bill Pay Category', 'Bill To']]
    df_drop_dup_line = df_select_line.drop_duplicates(subset = ['Facility', 'Bill To', 'Bill Pay Category'], 
                                                      keep = 'last')

    df_combine_header_line = pd.merge(df_drop_dup_line, df_drop_dup_header, how = 'left', 
                                      on = ['Facility', 'Bill To']).T.drop_duplicates().T
    df_combine_rename = df_combine_header_line.rename(columns = {"CustomerID_x": "CustomerID"})
    df_combine_week = df_combine_rename.reindex(columns = ['Accounting_CustomerID', 'CustomerID', 'DocNo', 'Facility', 
                                                                 'Bill Pay Category', 'Bill To', 'InvoiceDate'])
    print(df_combine_week.shape)
    
    df_list.append(df_combine_week)

(133, 7)
(637, 7)
(172, 7)
(199, 7)
(1132, 7)
(121, 7)
(576, 7)
(140, 7)
(1185, 7)
(136, 7)
(644, 7)
(159, 7)


### Generate Month Report with Accounting ID, Facility and  Bill Pay Category, etc.

In [4]:
df_combine_list = []
df_combine_month = df_list[0]

for df in range(len(df_list)):
    
    df_combine_month = pd.merge(df_combine_month, df_list[df], how = 'outer', 
                                  on = ['Accounting_CustomerID', 'CustomerID', 'DocNo', 
                                        'Facility', 'Bill Pay Category', 'Bill To'])
    print(df_combine_month.shape)
    
    df_combine_list.append(df_combine_month)
    
df_output_init = df_combine_list[-1]
df_output = df_output_init.T.drop_duplicates().T
df_output.rename({'InvoiceDate_x':'InvoiceDate', 'InvoiceDate_y':'InvoiceDate'}, axis = 'columns', inplace = True)

print(df_output.shape)

(133, 8)
(770, 9)
(942, 10)
(1141, 11)
(2273, 12)
(2394, 13)
(2970, 14)
(3110, 15)
(4295, 16)
(4431, 17)
(5075, 18)
(5234, 19)
(5234, 18)


In [61]:
ls1 = np.array(df_output.columns[0:6])

ls2 = np.array([])
for num in np.arange(1, 1 + len(df_output.columns[6:])):
    ls2 = np.append(ls2, str('Week ') + str(num))

week_name = np.append(ls1, ls2)
print(week_name)

df_output.columns = week_name

['Accounting_CustomerID' 'CustomerID' 'DocNo' 'Facility'
 'Bill Pay Category' 'Bill To' 'Week 1' 'Week 2' 'Week 3' 'Week 4'
 'Week 5' 'Week 6' 'Week 7' 'Week 8' 'Week 9' 'Week 10' 'Week 11'
 'Week 12']


In [54]:
df_output.head()

,Accounting_CustomerID,CustomerID,DocNo,Facility,Bill Pay Category,Bill To,Week 1,Week 2,Week 3,Week 4,Week 5,Week 6,Week 7,Week 8,Week 9,Week 10,Week 11,Week 12
0,SGCOMP0001,SGCOMP0001,19029476,Valley View,ACCESSORIAL,THE SG COMPANIES,01/09/2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,SGCOMP0001,SGCOMP0001,19029231,Valley View,ACCESSORIAL,THE SG COMPANIES,01/05/2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,CHRGEN0001,CHRGEN0001,19029455,Via Baron,ACCESSORIAL,C.H.ROBINSON-GENERAC VB,01/09/2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,CHRGEN0001,CHRGEN0001,19029455,Via Baron,HANDLING,C.H.ROBINSON-GENERAC VB,01/09/2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,CUBWOR0001,CUBWOR0001,19029404,New Jersey,HANDLING,CUBEWORK,01/08/2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Export Quarter Report As Excel File

In [55]:
# df_output.to_excel("01_03_2021 Weekly Report.xlsx", index = False, header = True)